In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Download dataset

In [ ]:
# Get mnist data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

## Helper functions


In [ ]:
# Init weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
# Init bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
# 2D Convolution
def conv2d(x,w):
    # x -> [batch,H,W,Channels]
    # w -> [filters H, filters W, Channels In, Channels Out]
    
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

In [ ]:
# Pooling layer
def max_pool_2by2(x):
    # x -> [batch,H,W,Channels]
    
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [ ]:
# Convolutional layer
def conv_layer(input_x,shape):
    
    w = init_weights(shape)   # Previus function to init weights 
    b = init_bias([shape[3]]) # Previus function to init bias
    
    return tf.nn.relu(conv2d(input_x,w)+b)

In [ ]:
# Fully connected layer
def fully_connected(input_layer,size):
    
    input_size = int(input_layer.get_shape()[1])
    
    w = init_weights([input_size,size]) # Previus function to init weights 
    b = init_bias([size])               # Previus function to init bias
    
    return tf.matmul(input_layer,w) + b

## Bulding the CNN

In [ ]:
# Placeholder
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

# Layers
x_image = tf.reshape(x,[-1,28,28,1]) # x -> [batch,H,W,Channels]

conv1 = conv_layer(x_image,shape=[5,5,1,32]) # w -> [filters H, filters W, Channels In, Channels Out]

pool1 = max_pool_2by2(conv1)

conv2 = conv_layer(pool1,shape=[5,5,32,64])

pool2 = max_pool_2by2(conv2)

conv2_flat = tf.reshape(pool2,[-1,7*7*64])

fully1 = tf.nn.relu(fully_connected(conv2_flat,1024))


hold_prob = tf.placeholder(tf.float32)

full_dropout = tf.nn.dropout(fully1,keep_prob=hold_prob)

y_pred = fully_connected(full_dropout,10)

# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

# Create tf session
init = tf.global_variables_initializer()

steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            
            print("On step {}".format(i))
            print("Accuracy: ")
            
            matches  = tf.equal(tf.arg_max(y_pred,1), tf.arg_max(y_true,1))
            
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            
            print('\n')
                  
            